In [1]:
import torch
import brt
from brt.common import log
# import brt.frontend
import brt.nn as nn
from brt.routers import TopKScatterRouter
log.set_level("primitive", "DEBUG")


class MoE(nn.Module):
    def __init__(self):
        super().__init__()
        self.scatter_router = TopKScatterRouter(route_num=2, gran_dim=10)
        self.expert1 = nn.Linear(10, 10)
        self.expert2 = nn.Linear(10, 10)

    def forward(self, x):
        route_results, reverse_indice, reverse_shape = self.scatter_router(x)
        x_0 = self.expert1(route_results[0])
        x_1 = self.expert2(route_results[1])
        x = x_0 + x_1
        return x


class MoEModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.moe = MoE()

    def forward(self, x):
        return self.moe(x)


moe_model = MoEModel()

x = torch.ones((60, 10))
x = x.cuda()
moe_model.cuda()
z = moe_model(x)
print(z)


DEBUG:brainstorm.primitive:using brt_forward
DEBUG:brainstorm.primitive:using brt_forward
tensor([ 0.3787,  0.5939, -0.0650, -0.6993, -1.6051,  0.6451,  0.2829, -0.8155,
         0.2437,  0.2173], device='cuda:0', grad_fn=<AddBackward0>)
